In this exercise we explore $K$-means clustering - and we it out on the locations of the `PROSTITUTION` crime type. Applying a clustering method makes sense because we know from our earlier work that this crime type tends to happen in only a few locations. We'll also talk a little bit about model selection and [overfitting](https://www.youtube.com/watch?v=DQWI1kvmwRg) in unsupervised models.

> _Exercise_: $K$-means
> 
> * Visualize the prostitution data (e.g. by plotting it on a map)
> * Train models of $K = 2,\ldots,10$ on the prostitution data.
> * Explore how the total squared error changes as a function of $K$ and identify what you think is the right number of clusers based on the knee-point in the squared error plot.
> * And by the way: The fit only gets better when we add more means - why not keep adding more of them: Explain in your own words why it makes sense to stop around a knee-point.
> * Another way of estimating the right number of clusters in a $K$-means problem is _stability analysis_. The idea is the following
>   - For each $K = 2,\ldots,10$ generate $N = 10$ clusterings based on random 50% of data (or some other fraction of data/bootstrap).
>   - Calculate the pairwise similarity between the clusterings. 
>   - We now define _stability_ for some value of $K$ as average pairwise similarity of the $N$ clustering, where the similarity is the cosine distance $\frac{\mathbf{c}_i^K\cdot\mathbf{c}_j^K}{||\mathbf{c}_i^K||\,||\mathbf{c}_j^K||}$ between centroid vectors $\mathbf{c}_i^K$ and $\mathbf{c}_j^K$.
>   - We now say that the right $K$ maximizes stability.
> * Explain why stability should help you find the right number of clusters.
> * **Optional**: Perform stability analysis on the prostitution data. 

In [4]:
from csv import reader
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
import collections

import matplotlib.pylab as plt
import geoplotlib
from geoplotlib.utils import BoundingBox, DataAccessObject

data = open("SFPD_Incidents_-_from_1_January_2003.csv")
next(data,1).split(',')

category = []
description = []
day_of_week = []
date = []
time = []
police_district = []
lon = []
lat = []

for line in reader(data): 
    var1, var2, var3, var4, var5, var6, var7, var8, var9, var10, var11, var12, var13 = line
    
    category.append(var2)
    description.append(var3)
    day_of_week.append(var4)
    date.append(var5)
    time.append(var6)
    police_district.append(var7)
    lon.append(float(var10))
    lat.append(float(var11))

#Bounding box
eps_center = 0.05
max_lat = np.mean(lat)+eps_center
min_lat = np.mean(lat)-eps_center
max_lon = np.mean(lon)+eps_center
min_lon = np.mean(lon)-eps_center
bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)

lat = np.asarray(lat).astype(np.float)
lon = np.asarray(lon).astype(np.float)

focuscrimes = ['PROSTITUTION', 'DRUG/NARCOTIC', 'DRIVING UNDER THE INFLUENCE']


# Find incidents in the district
for crime in focuscrimes:
    mask = np.zeros_like(category,dtype='bool')
    for i in range(len(category)):
        if category[i] == 'PROSTITUTION':
            mask[i] = True
    data = {'lat': lat[mask], 'lon': lon[mask]}
    
geoplotlib.dot(data)
geoplotlib.set_bbox(bbox)
geoplotlib.show()


* The code above is a visulisation of the prostitution data.

In [ ]:
from geoplotlib.colors import create_set_cmap
import pyglet
from sklearn.cluster import KMeans
import geoplotlib
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.utils import BoundingBox
import numpy as np

class KMeansLayer(BaseLayer):
    

    def __init__(self, data):
        self.data = data
        self.k = 2


    def invalidate(self, proj):
        self.painter = BatchPainter()
        x, y = proj.lonlat_to_screen(self.data['lon'], self.data['lat'])

        k_means = KMeans(n_clusters=self.k)
        k_means.fit(np.vstack([x,y]).T)
        labels = k_means.labels_

        self.cmap = create_set_cmap(set(labels), 'hsv')
        for l in set(labels):
            self.painter.set_color(self.cmap[l])
            #Enabling the line below enables the convexhull algorithm.
            #self.painter.convexhull(x[labels == l], y[labels == l])
            self.painter.points(x[labels == l], y[labels == l], 5)
    
            
    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        ui_manager.info('Use left and right to increase/decrease the number of clusters. k = %d' % self.k)
        self.painter.batch_draw()


    def on_key_release(self, key, modifiers):
        if key == pyglet.window.key.LEFT:
            self.k = max(2,self.k - 1)
            return True
        elif key == pyglet.window.key.RIGHT:
            self.k = self.k + 1
            return True
        return False
  

geoplotlib.add_layer(KMeansLayer(data))
geoplotlib.set_smoothing(True)
geoplotlib.set_bbox(bbox)
geoplotlib.show()


* Above is the Train model from 2 to 10 (or above), since K in this program can be increased by using the arrorw keys.

* I belive the right number of clusters is found by starting with one cluster (k=1) and then keep dividing clusters until the points assigned to each cluster have a Gaussian distribution.

* The reason why we are not interested in continously adding more means is because if you keep splitting to 'infinitity', you'll just end up with a cluster around every single item in your data set.

* Stability analysis should help us find the right number of cluters due to cosine distance comparison of clusters. We know that the if cosine distance is equal to 1 then the similarity between clusters has reached its maximum stability. It should be mentioned that reaching 1 is not nessecarily possible, but as close to 1 as possible will surfice. 